In [16]:
import numpy as np
import sympy as sp

In [17]:
from itertools import chain, combinations

def powerset(iterable):
    s = list(iterable)
    return list(map(list, chain.from_iterable(combinations(s, r) for r in range(len(s)+1))))

In [18]:
def submat(mat, rs, cs):
        return mat[np.ix_(rs, cs)]

def minor(mat, rs, cs):
        return np.round(np.linalg.det(submat(mat, rs, cs)))
    
def startMinor(mat, i, j):
        size = 0

        if i < j:
            size = i
        else:
            size = j
        
        rs = list(range(i - size, i + 1))
        cs = list(range(j - size, j + 1))

        return minor(mat, rs, cs)

def totallyPositive(mat):
        for i in range(mat.shape[0]):
            for j in range(mat.shape[1]):
                if startMinor(mat, i, j) <= 0:
                    return False
        return True
    
def totallyNonnegative(mat):
    n = mat.shape[0]
    combs = powerset(list(range(n)))
    combs = [(c1, c2) for c1 in combs for c2 in combs if len(c1) == len(c2)]
    
    for rs, cs in combs:
            if minor(mat, rs, cs) < 0:
                return False
    return True


In [19]:
import pandas as pd

def sortThrough(N, D, mat, table, dep=1, i=0, j=0, side='right'):
    for e in range(1, D + 1):
        mat[i, j] = e
        if startMinor(mat, i, j) <= 0:
            continue

        if i == N - 1 and j == N - 1:
            for k in range(np.max(mat),  D + 1):
                table[k] += 1
            continue

        if side == 'right':
            if j == N - 1:
                table = sortThrough(N, D, mat, table, dep, dep, dep - 1, 'down')
            else:
                table = sortThrough(N, D, mat, table, dep, i, j + 1, 'right')
        if side == 'down':
            if i == N - 1:
                table = sortThrough(N, D, mat, table, dep + 1, dep, dep, 'right')
            else:
                table = sortThrough(N, D, mat, table, dep, i + 1, j, 'down')
    return table

table = pd.DataFrame(columns=[1, 2, 3, 4, 5, 6, 7])

for n in range(1, 7):
    d = 7
    if n == 4:
        d = 5
    if n == 5:
        d = 4
    if n == 6:
        d = 3

    table_d = { i : 0 for i in range(1, d + 1) }
    table_d = sortThrough(n, d, np.zeros((n, n), dtype=int), table_d)
    table = pd.concat([table, pd.DataFrame([table_d])], ignore_index=True)
    print(f'Size {n} done!')

table.index += 1
table

Size 1 done!
Size 2 done!
Size 3 done!
Size 4 done!
Size 5 done!
Size 6 done!


,1,2,3,4,5,6,7
1,1,2,3,4,5,6,7
2,0,5,33,112,288,605,1145
3,0,0,22,597,7178,43090,207494
4,0,0,0,0,156,NaN,NaN
5,0,0,0,0,NaN,NaN,NaN
6,0,0,0,NaN,NaN,NaN,NaN
